In [ ]:
# Desenvolvimento de ACs para a simulação da propagação de notícias, considerando NOVAS, VELHAS e SEM notícia (e ESQUECIMENTO)

# Importar as bibliotecas utilizadas no decorrer do programa:
from tomato.classes import cell
from matplotlib import pyplot as plt
import numpy as np

# Determinação das regras de transição:
class NewsPropagation(cell.CellTemplate):
    
    # Matriz utilizada para armazenar o número de gerações que passaram, desde que as células adquiriram um dado estado
    generation_matrix = np.ones((100, 100))
    
    def update(self, state_matrix):
        
        self.state_matrix = state_matrix
        
        # Número de gerações necessárias para que a célula esqueça a informação
        gens_forget = 10
        
        # p_2_alone: probabilidade de esquecimento, para célula SOZINHA
        p_2_alone = 0.85
        
        # Número máximo de células (com informação) na vizinhança, para que seja considerada sozinha
        R_alone = 2
        
        # p_2_company: probabilidade de esquecimento, para célula ACOMPANHADA
        p_2_company = 0.40
        
        # p2: variável que define a probabilidade de esquecimento
        p2 = np.random.uniform(0, 1)
 
        # p_1: probabilidade que a informação seja propagada
        p_1 = np.random.normal(0, 0.65)
        
        # R: threshold
        R = 1.05
       
        if self.value == -1:
            
            if self.information_accepted_new < 3:
                    p_1 = p_1*1.5
             
            # Para que a célula adquira informação, m*p_1 >= R
            if (self.information_accepted_new*p_1) >= R:
                    self.value = 1
                    NewsPropagation.generation_matrix[self.pos[0], self.pos[1]] = 0
                    
            else:
                self.value = -1

        if self.value == 1:
            
            if (NewsPropagation.generation_matrix[self.pos[0], self.pos[1]]) > (gens_forget):
                
                # Célula SOZINHA:
                if self.information_accepted_new <= R_alone:
                    if p2 <= p_2_alone:
                        self.value = 0
                        NewsPropagation.generation_matrix[self.pos[0], self.pos[1]] = 0
                    else: 
                        self.value = 1
                        
                # Célula ACOMPANHADA:
                else:
                    if p2 <= p_2_company:
                        self.value = 0
                        NewsPropagation.generation_matrix[self.pos[0], self.pos[1]] = 0
                    else:
                        self.value = 1
            else:
                self.value = 1
                         
        if self.value == 0:
            # Se não há células SEM informação na vizinhança, torna-se SEM informação
            if self.information_not_accepted == 0:
                self.value = -1
                
        NewsPropagation.generation_matrix[self.pos[0], self.pos[1]] += 1
        
            
    
    @property
    def neighbors(self):
        return self.moore_neighborhood
        # Considera-se a vizinhança de MOORE

    @property
    def information_accepted_new(self):
        return self.neighbors.count(1)
        # Conta o n° de vizinhos com informação NOVA
        
    @property
    def information_accepted_old(self):
        return self.neighbors.count(0)
        # Conta o n° de vizinhos com informação VELHA

    @property
    def information_not_accepted(self):
        return self.neighbors.count(-1)
        # Conta o n° de vizinhos SEM informação

    @staticmethod
    def display(value):
        if value == -1:
            return (0, 0, 0)
        # Células SEM informação

        elif value == 0:
            return (128, 128, 128)
        # Células com informações ANTIGAS
        
        elif value == 1:
            return (255, 255, 255)
        # Células com informações NOVAS

    @staticmethod
    def from_display(value):
        if (value == (0, 0, 0)).all():
            return -1
        
        elif (value == (128, 128, 128)).all():
            return 0
        
        else:
            return 1

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = NewsPropagation

CELL_SIZE = 1
dimensions = (100, 100)

state_matrix = np.random.choice(a=[-1, 0, 1], size = dimensions, p=[0.80, 0.05, 0.15])

initial_state_matrix = state_matrix

board = tt.Board(rule, cell_size=CELL_SIZE)
board.load_state(initial_state_matrix)

# Criando vetores numpy para armazenar as populações de cada espécie de célula
news_new_pop = np.zeros(100)
news_old_pop = np.zeros(100)
no_news_pop = np.zeros(100)

while board.generation < 100:
    # Matriz com os estados de cada célula:
    state_matrix = board.state_matrix

    news_new_pop[board.generation] = np.count_nonzero(state_matrix == 1)
    news_old_pop[board.generation] = np.count_nonzero(state_matrix == 0)
    no_news_pop[board.generation] = np.count_nonzero(state_matrix == -1)

    # Iterar a simulação depois de terminada a nossa coleta de dados da geração
    board.update()

# Avisando que a simulação acabou e fazendo umas estatísticas básicas
print("Prontinho!")
print(f"news_new_pop: initial {news_new_pop[0]} | final {news_new_pop[-1]} | avg {np.mean(news_new_pop)}")
print(f"news_old_pop: initial {news_old_pop[0]} | final {news_old_pop[-1]} | avg {np.mean(news_old_pop)}")
print(f"no_news_pop: initial {no_news_pop[0]} | final {no_news_pop[-1]} | avg {np.mean(no_news_pop)}")

initial_news_new = news_new_pop[0]/(news_new_pop[0] + no_news_pop[0] + news_old_pop[0])
initial_news_old = news_old_pop[0]/(news_new_pop[0] + no_news_pop[0] + news_old_pop[0])
initial_no_news = no_news_pop[0]/(news_new_pop[0] + no_news_pop[0] + news_old_pop[0])

final_news_new = news_new_pop[-1]/(news_new_pop[-1] + no_news_pop[-1] + news_old_pop[-1])
final_news_old = news_old_pop[-1]/(news_new_pop[-1] + no_news_pop[-1] + news_old_pop[-1])
final_no_news = no_news_pop[-1]/(news_new_pop[-1] + no_news_pop[-1] + news_old_pop[-1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(news_new_pop, color="red", label="New News")
ax.plot(news_old_pop, color="green", label="Old News")
ax.plot(no_news_pop, color="blue", label="No news")

ax.set_title("Evolução das populações")
ax.set_xlabel("Geração")
ax.set_ylabel("População")
ax.legend()

In [ ]:
mycolors = ["red", "green", "blue"]
mylabels = ["New news", "Old news", "No news"]
myexplode = [0.05, 0.05, 0.05]

# Populações iniciais:
y = np.array([initial_news_new, initial_news_old, initial_no_news])
plt.title("Populações iniciais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show()

# Populações finais:
y = np.array([final_news_new, final_news_old, final_no_news])
plt.title("Populações finais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show() 

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = NewsPropagation

CELL_SIZE = 1
dimensions = (100, 100)

state_matrix = np.random.choice(a=[-1, 0, 1], size = dimensions, p=[0.80, 0.05, 0.15])

initial_state_matrix = state_matrix

board = tt.Board(rule, cell_size=CELL_SIZE)
board.load_state(initial_state_matrix)

board.start(
    state_matrix, 
    inline = True, 
    generations = 100, 
    generate_figures = True, 
    generate_figures_dir = "Trabalho_Final", # Especificar o nome da pasta
    generate_gif = True # Criar um gif com as imagens ao final da execução
)

board.save_png("Modelo_3_1_GIF.png")